# Orbax v0 to v1 Migration Guide

## How to use v1 API to load checkpoints saved with v0 API

v1 `ocp.load_*` API can load checkpoints saved with v0 API. But before discussing the details, let us first understand how saved checkpoints are laid out.

### Checkpoint Layouts

#### Checkpointables in subdirectories

Most commonly, Orbax saves a checkpoint in a directory, which in turn contains subdirectories containing checkpointables (items).

e.g. The checkpoint in **step_1234** contains checkpointables in subdirectories named as **state** and **my_json_data**.

```
root_dir/
    step_1234/
        _CHECKPOINT_METADATA
        state/
            _METADATA
            manifest.ocdbt
            ocdbt.process_0/
        pytree/
            _METADATA
            manifest.ocdbt
            ocdbt.process_0/
        my_json_data/
            my_data.json
```
A `CheckpointManager` pointing to `root_dir/` saves checkpoints for each step in the above format.

Similarly, `Checkpointer(CompositeCheckpointHandler)` can save a checkpoint like `step_1234/`, though the directory can be arbitrary (not constrained to correspond to a specific step).

Let's save a checkpoint with the V0 API to demonstrate.

In [1]:
# Save checkpoint with checkpointables in state and pytree subdirs.

from etils import epath
import numpy as np
from orbax import checkpoint as ocp_v0

root_dir = epath.Path('/tmp/migration/root_dir')
root_dir.rmtree(missing_ok=True)  # Clean up if it already exists.
data = {
  'params': np.ones(2),
}

args = ocp_v0.args.Composite(**{
  checkpointable_name: ocp_v0.args.StandardSave(data)
  for checkpointable_name in ['state', 'pytree']
})
with ocp_v0.CheckpointManager(root_dir) as mngr:
  step = 0
  mngr.save(step, args=args)
  
step_dir = root_dir / f'{step}'


INFO:2025-07-18 10:11:04,772:jax._src.xla_bridge:821: Unable to initialize backend 'pathways': Could not initialize backend 'pathways'
INFO:2025-07-18 10:11:04,773:jax._src.xla_bridge:821: Unable to initialize backend 'proxy': INVALID_ARGUMENT: IFRT proxy server address must be '<transport-type>://<backend-address>' (e.g., 'grpc://localhost'), but got 
INFO:2025-07-18 10:11:04,783:jax._src.xla_bridge:821: Unable to initialize backend 'mlcr': Could not initialize backend 'mlcr'
INFO:2025-07-18 10:11:04,784:jax._src.xla_bridge:821: Unable to initialize backend 'sliceme': Could not initialize backend 'sliceme'


In [4]:
!ls /tmp/migration/root_dir/0

_CHECKPOINT_METADATA
pytree
state


A checkpoint stored in above layout can be loaded using `ocp.load_checkpointables(...)` function.

In [5]:
# Load all checkpointables from a directory where subdirs contain checkpointables.
import orbax.checkpoint.experimental.v1 as ocp

loaded = ocp.load_checkpointables(step_dir)
# Use the checkpointables.
state = loaded['state']
pytree = loaded['pytree']

print('state=', state)
print('pytree=', pytree)

state= {'params': array([1., 1.])}
pytree= {'params': array([1., 1.])}


#### Checkpoint in directory with no subdirectory

Alternatively, users can save checkpoints directly to a directory without any checkpointables (subdirectory).

e.g. The following layout contains an pytree checkpoint without any names like `state` as above.
```
my_checkpoint/
        _CHECKPOINT_METADATA
        _METADATA
        manifest.ocdbt
        ocdbt.process_0/
```
v0 `Checkpointer` (without `CompositeCheckpointHandler`) can be used to save in such layouts.

In [6]:
# Save a checkpoint directly to a directory.

my_checkpoint_dir = epath.Path('/tmp/migration/custom_checkpoint/my_checkpoint')
my_checkpoint_dir.rmtree(missing_ok=True)

with ocp_v0.StandardCheckpointer() as checkpointer:
  checkpointer.save(my_checkpoint_dir, data)

In [7]:
!ls /tmp/migration/custom_checkpoint/my_checkpoint

_CHECKPOINT_METADATA
d
descriptor
manifest.ocdbt
_METADATA
ocdbt.process_0


An pytree checkpoint in the above layout can be loaded using `ocp.load_pytree(...)` function.

In [8]:
# Load a pytree from a directory with no checkpointables.

loaded = ocp.load_pytree(my_checkpoint_dir, checkpointable_name=None)
# Use the loaded pytree.
print('loaded=', loaded)

loaded= {'params': array([1., 1.])}


### Compatibility Matrix

#### Loading pytree checkpoint with `load_pytree(...)`

| Restore API | Response
:------- | :-------- |
|ocp.load_pytree(`step_1234`)|Loads PyTree under subdirectory, `pytree`|
|ocp.load_pytree(`step_1234`, `checkpointable_name='pytree'`)|Loads PyTree under subdirectory, `pytree`|
|ocp.load_pytree(`step_1234`, `checkpointable_name='state'`)|Loads PyTree under subdirectory, `state`|
|ocp.load_pytree(`my_checkpoint`, `checkpointable_name=None`)|Loads PyTree directly from `my_checkpoint`|

Following calls will lead to error.

| Restore API | Response
:------- | :-------- |
|ocp.load_pytree(`root_dir`)|Error: expecting a subdir named `pytree`|
|ocp.load_pytree(`root_dir`, `checkpointable_name='pytree'`)|Error: expecting a subdir named `pytree`|
|ocp.load_pytree(`root_dir`, `checkpointable_name=None`)|Error: expecting pytree metadata file|
|ocp.load_pytree(`step_1234`, `checkpointable_name=None`)|Error: expecting pytree metadata file|
|ocp.load_pytree(`my_checkpoint`)|Error: expecting a subdir named `pytree`|
|ocp.load_pytree(`my_checkpoint`, `checkpointable_name='pytree'`)|Error: expecting a subdir named `pytree`|

#### Loading checkpointables with `load_checkpointables(...)`

| Restore API | Response
:------- | :-------- |
|ocp.load_checkpointables(`step_1234`)|Loads all checkpointables from respective subdirs|
|ocp.load_checkpointables(`step_1234`, `dict(state=abstract_tree, my_json_data=None)`)|Loads `state` and `my_json_data` checkpointables from respective subdirs|

Following calls will lead to error.

| Restore API | Response
:------- | :-------- |
|ocp.load_checkpointables(`root_dir`)|Error: suggesting to try a subdir instead|
|ocp.load_checkpointables(`my_checkpoint`)|Error: suggesting to use load_pytree instead|
|ocp.load_checkpointables(`root_dir`, `dict(state=abstract_tree, pytree=abstract_tree)`)|Error: suggesting to try a subdir instead|
|ocp.load_checkpointables(`my_checkpoint`, `dict(state=abstract_tree, pytree=abstract_tree)`)|Error: suggesting to use load_pytree instead|


## Migrating from v0 CheckpointManager to v1 Checkpointer

If you were using `v0 CheckpointManager` in your training loop then switch to `v1
Checkpointer`.

Please consult the following table for complete list of compatible methods.

| v0 CheckpointManager | v1 Checkpointer
:------- | :-------- |
|`directory`|`directory`|
|`all_steps(...)`|`checkpoints`|
|`latest_step()`|`latest`|
|`reload()`|`reload()`|
|`should_save(step)`|`should_save(step)`|
|`save(...)`|`save_pytree(...)`, `save_checkpointables(...)`|
||and `save_*_async(...)`|
|`restore(...)`|`load_pytree(...)`, `load_checkpointables(...)` |
||and `load_*_async(...)`|
|`item_metadata(step)`|`pytree_metadata(step)`,|
||`checkpointables_metadata(step)`|
|`metrics(step)`|`pytree_metadata(step).metrics`,|
||`checkpointables_metadata(step).metrics`|
|`metadata(step)`|`pytree_metadata(step)`,|
||`checkpointables_metadata(step)`|
|`metadata(None)` or `metadata()`|`root_metadata()`|
|`wait_until_finished`|Call `AsyncResponse.result()`|
||returned from `save_*_async(...)` and `load_*_async(...)`.|
|`check_for_errors()`|Call `AsyncResponse.result()`|
||returned from `save_*_async(...)` and `load_*_async(...)`.|
|`close()`|`close()`|
|`is_saving_in_progress()`|`is_saving_in_progress()`|
|`best_step()`|`checkpoints.metrics.best_step`|
|`reached_preemption(...)`|Unsupported|
|`delete(step)`|Coming soon...|
